In [ ]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!pip install bs4
!pip install requests
! pip install kaggle

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
chromium-chromedriver is already the newest version (1:85.0.4183.83-0ubuntu2.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 36 n

In [ ]:
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import pandas as pd
import datetime
import time
import matplotlib.pyplot as plt
import json
from bs4 import BeautifulSoup
import requests
from urllib.parse import urljoin
from PIL import Image
import tempfile
import numpy as np

In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d saurav9786/amazon-product-reviews

 92% 101M/109M [00:00<00:00, 158MB/s] 
100% 109M/109M [00:00<00:00, 129MB/s]


In [ ]:
! unzip amazon-product-reviews.zip

Archive:  amazon-product-reviews.zip
  inflating: ratings_Electronics (1).csv  


In [ ]:
df=pd.read_csv("ratings_Electronics (1).csv",names=['user_id', 'product_id','rating','timestamp'])

In [ ]:
df

,user_id,product_id,rating,timestamp
0,AKM1MP6P0OYPR,0132793040,5.0,1365811200
1,A2CX7LUOHB2NDG,0321732944,5.0,1341100800
2,A2NWSAGRHCP8N5,0439886341,1.0,1367193600
3,A2WNBOD3WNDNKT,0439886341,3.0,1374451200
4,A1GI0U4ZRJA8WN,0439886341,1.0,1334707200
...,...,...,...,...
7824477,A2YZI3C9MOHC0L,BT008UKTMW,5.0,1396569600
7824478,A322MDK0M89RHN,BT008UKTMW,5.0,1313366400
7824479,A1MH90R0ADMIK0,BT008UKTMW,4.0,1404172800
7824480,A10M2KEFPEQDHN,BT008UKTMW,4.0,1297555200


In [ ]:
#nombre d'articles évalués par chaque utilisateur
user_article_counts = df.groupby('user_id')['product_id'].nunique()

In [ ]:
print(user_article_counts)

user_id
A00000262KYZUE4J55XGL    1
A000063614T1OE0BUSKUT    2
A00009182QVLSWIGHLS1B    1
A00009661LC9LQPGKJ24G    1
A00010809P09NUU6ZP6H     1
                        ..
AZZZOVIBXHGDR            3
AZZZRS1YZ8HVP            2
AZZZSIK7NFFVP            1
AZZZWXXUPZ1F3            1
AZZZY1W55XHZR            1
Name: product_id, Length: 4201696, dtype: int64


In [ ]:
#les 1000 utilisateurs ayant évalué plus de 50 articles (car il n y'a pas 1000 utilisateur ayant évalué plus de 100)
selected_users = user_article_counts[user_article_counts > 50].sample(n=1000, replace=False).index

print(len(selected_users))

1000


In [ ]:
sample1_df = df[df['user_id'].isin(selected_users)]


In [ ]:
sample1_df

,user_id,product_id,rating,timestamp
118,AT09WGFUM934H,0594481813,3.0,1377907200
492,A3CLWR1UUZT6TG,0972683275,5.0,1373587200
631,A3TAS1AG6FMBQW,0972683275,5.0,1353456000
1322,A1A5KUIIIHFF4U,1400501466,1.0,1332547200
1456,A1E3OB6QMBKRYZ,1400501466,1.0,1350086400
...,...,...,...,...
7824414,A2XX2A4OJCDNLZ,B00LED02VY,4.0,1405123200
7824423,A34BZM6S9L7QI4,B00LGQ6HL8,5.0,1405555200
7824424,A1G650TTTHEAL5,B00LGQ6HL8,5.0,1405382400
7824425,A25C2M3QF9G7OQ,B00LGQ6HL8,5.0,1405555200


In [ ]:
# Compter le nombre de product_id uniques dans le dataset
unique_product_ids = sample1_df['product_id'].nunique()
print("Nombre de product_id uniques :", unique_product_ids)

Nombre de product_id uniques : 36787


In [ ]:
# Compter le nombre de product_id uniques dans le dataset
unique_user_ids = sample1_df['user_id'].nunique()
print("Nombre de user_id uniques :", unique_user_ids)

Nombre de user_id uniques : 1000


In [ ]:
df_to_scrap = sample1_df.drop_duplicates(subset='product_id')


In [ ]:
df_to_scrap

,user_id,product_id,rating,timestamp
118,AT09WGFUM934H,0594481813,3.0,1377907200
492,A3CLWR1UUZT6TG,0972683275,5.0,1373587200
1322,A1A5KUIIIHFF4U,1400501466,1.0,1332547200
1990,A1PPS91NLI7KEH,1400532620,5.0,1279843200
2150,A3CLWR1UUZT6TG,1400532655,5.0,1291420800
...,...,...,...,...
7824355,A3A4ZAIBQWKOZS,B00LA6T0LS,4.0,1405036800
7824396,A3NHUQ33CFH3VM,B00LBZ1Z7K,3.0,1405814400
7824414,A2XX2A4OJCDNLZ,B00LED02VY,4.0,1405123200
7824423,A34BZM6S9L7QI4,B00LGQ6HL8,5.0,1405555200


In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(chrome_options)

In [ ]:
product_scrapped = []
description_list = " "
df_result = pd.DataFrame(columns=['product_id', 'product_name', 'image_URL', 'description'])

In [ ]:
racine_URL = 'https://www.amazon.com/dp/'

In [ ]:
i=1
for index, row in df_to_scrap.iterrows():

  product_id = row['product_id']
  # Vérifier si le product_id est déjà scrapé
  if product_id in product_scrapped:
    #print(f"Product ID {product_id} déjà scrapé. Passer au produit suivant.")
    continue

  URL = racine_URL + str(product_id)

  driver.get(URL) #enter the website

  # Attendre quelques secondes pour le chargement complet de la page (peut être ajusté)
  time.sleep(5)
  # Récupérer le contenu de la page après le chargement dynamique
  page_source = driver.page_source
  # Utiliser Beautiful Soup pour analyser le contenu
  soup = BeautifulSoup(page_source, 'html.parser')

  print(f"Article {i} id : {product_id}:")
      # Extraction du nom du produit

  try:

    title = soup.find('div', id='title_feature_div')
    titre = title.find('span')
    nom_objet = titre.text.strip()

    #print(f'Nom du produit : {nom_objet}')
    #print('--')

    # Extraction de l'URL de l'image
    img_tag = soup.find('img', class_='a-dynamic-image')
    if img_tag:
      img_url = img_tag.get('src')
    else:
      img_url = None
    #print(f'URL image : {img_url}')
    #print('--')


      #Extraction de la description du produit
    #print('Description :')
    description_container = soup.find('div', id='featurebullets_feature_div')
    values = description_container.find_all('span')
    for value in values :
      description = value.text.strip()
      description_list = description_list + description


    #print(description_list)
    #data = {'product_id': product_id, 'product_name': nom_objet, 'image_URL': img_url, 'description': description_list}
    #df_result = df_result.append(data, ignore_index=True)
    data = pd.DataFrame({'product_id': [product_id], 'product_name': [nom_objet], 'image_URL': [img_url], 'description': [description_list]})
    df_result = pd.concat([df_result, data], ignore_index=True)

    description_list = " "



    product_scrapped.append(product_id)
  except Exception as e:
    #print(f"Page HTML non existante {product_id}: {str(e)}")
    i=i+1
    product_scrapped.append(product_id)
    print("-" * 50)
    continue


  print("-" * 50)
  i=i+1

Article 1 id : 0594481813:
--------------------------------------------------
Article 2 id : 0972683275:
--------------------------------------------------
Article 3 id : 1400501466:
--------------------------------------------------
Article 4 id : 1400532620:
--------------------------------------------------
Article 5 id : 1400532655:
--------------------------------------------------
Article 6 id : 1400599997:
--------------------------------------------------
Article 7 id : 1400698987:
--------------------------------------------------
Article 8 id : 1400699169:
--------------------------------------------------
Article 9 id : 787988002X:
--------------------------------------------------
Article 10 id : 8862935307:
--------------------------------------------------
Article 11 id : 8862936826:
--------------------------------------------------
Article 12 id : 9043413585:
--------------------------------------------------
Article 13 id : 9573212919:
---------------------------------

KeyboardInterrupt: 

In [ ]:
len(product_scrapped)

0

In [ ]:
df_result

,product_id,product_name,image_URL,description
0,1400501466,"Barnes & Noble NOOK Tablet 16gb (Color, BNTV250)",https://m.media-amazon.com/images/I/61XyscllBu...,NOOK (Color) TabletTM - Nov. 2011 - BNTV250HD...
1,1400532620,Fujimoto Dengyo Barnes and Noble Nook eBook Re...,https://m.media-amazon.com/images/I/61DGW-XwVC...,This refurbished product is tested and certif...
2,1400532655,Barnes & Noble Nook Color eReader,https://m.media-amazon.com/images/I/416vS-HMwK...,"Get social, surf the web, play games, even li..."
3,1400599997,Barnes & Noble NOOK ebook reader (WiFi + 3G)[B&W],https://m.media-amazon.com/images/I/91+vyqcLMu...,Barnes & Noble brand ebook reader: NookE Ink ...
4,1400699169,Barnes & Noble NOOK HD+ Tablet 32GB Slate (BNT...,https://m.media-amazon.com/images/I/511kFogV6G...,This refurbished product is tested and certif...
5,787988002X,Micro Connector for your Cellular Phone Model ...,https://m.media-amazon.com/images/I/51sZM0GwZg...,
6,8862935307,Moleskine® Kindle DX Cover with Reporter-Style...,https://m.media-amazon.com/images/I/91uYialKmf...,"Signature simplicity and style, made exclusiv..."
7,8862936826,Moleskine Folio Digital Tablet Cover for Apple...,https://m.media-amazon.com/images/I/61L5HRjIGa...,CARRY & PROTECT YOUR DIGITAL TOOLS: Moleskine...
8,9573212919,External USB DVD/CD,https://m.media-amazon.com/images/I/71ld6rxnk9...,"USB CD/DVD External Optical DriveCD Text, CD ..."
9,9800359788,TP-Link 5-Port Gigabit Desktop Switch,https://m.media-amazon.com/images/I/51iWQRu6eB...,TP-LINK TL-SG 1005 D 5-port Gigabit Switch
